In [1]:
#安装 libsndfile
'''
cd /home/ec2-user/SageMaker/shouqi/libsndfile-1.0.28
./configure --prefix=/usr    \
            --disable-static \
            --docdir=/usr/share/doc/libsndfile-1.0.28 &&
make && sudo make install
'''
# !pip install asteroid -i https://pypi.tuna.tsinghua.edu.cn/simple/
# !pip install librosa -i https://pypi.tuna.tsinghua.edu.cn/simple/

import os
import subprocess
import sagemaker
import torch
from sagemaker import get_execution_role
from helper.utils import *
import argparse
import librosa
import torch
from scipy.signal import lfilter
import numpy as np
import scipy.io.wavfile
import soundfile as sf
import fnmatch, os, warnings
import json
from code.pred_utils import *

sagemaker_session = sagemaker.Session()
_role = get_execution_role()

s3_path='s3://sagemaker-cn-northwest-1-969422986683/shouqi' #上传路径
sample_rate=16000 #采样率


## 准备数据

In [4]:
def generate_data(in_dir,out_dir,out_filename,ttype): #生成array以及meta_json
    file_infos = []
    in_dir = os.path.abspath(in_dir)
    wav_list = os.listdir(in_dir)
    for wav_file in wav_list:
        if not wav_file.endswith('.wav'):
            continue
        wav_path = os.path.join(in_dir, wav_file)
        samples, _ = librosa.load(wav_path, sr=sample_rate)
        fake_wav_path = os.path.join('/opt/ml/input/data', ttype, out_filename, wav_file)
        file_infos.append((fake_wav_path, len(samples)))
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    with open(os.path.join(out_dir, out_filename + '.json'), 'w') as f:
        json.dump(file_infos, f, indent=4)
        
generate_data('train_sample_v6/tr/mix_both','train_sample_v6/tr','mix_both','train')
generate_data('train_sample_v6/tr/s1','train_sample_v6/tr','s1','train')
generate_data('train_sample_v6/tr/s2','train_sample_v6/tr','s2','train')
generate_data('train_sample_v6/cv/mix_both','train_sample_v6/cv','mix_both','test')
generate_data('train_sample_v6/cv/s1','train_sample_v6/cv','s1','test')
generate_data('train_sample_v6/cv/s2','train_sample_v6/cv','s2','test')


!aws s3 sync train_sample_v6/tr/ $s3_path/separate/train
!aws s3 sync train_sample_v6/cv/ $s3_path/separate/test

s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/train
upload: train_sample_v6/tr/mix_both.json to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/train/mix_both.json
upload: train_sample_v6/tr/mix_both/1-35-t-s-n-4_1-46-nav-n-2_0dB.wav to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/train/mix_both/1-35-t-s-n-4_1-46-nav-n-2_0dB.wav
upload: train_sample_v6/tr/mix_both/1-46-t-s-n-1_1-40-nav-n-2_0dB.wav to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/train/mix_both/1-46-t-s-n-1_1-40-nav-n-2_0dB.wav
upload: train_sample_v6/tr/mix_both/1-40-t-n-1_1-37-nav-n-1_0dB.wav to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/train/mix_both/1-40-t-n-1_1-37-nav-n-1_0dB.wav
upload: train_sample_v6/tr/mix_both/1-23-t-m-n-1_1-47-nav-n-3_0dB.wav to s3://sagemaker-cn-northwest-1-969422986683/shouqi/separate/train/mix_both/1-23-t-m-n-1_1-47-nav-n-3_0dB.wav
upload: train_sample_v6/tr/mix_both/1-40-t-n-1_1-41-nav-n-1_0dB.wav to s3://sagemaker-cn-nor

## 开始训练和部署流程
### 设置训练和验证数据路径

In [2]:
train_inputs="{}/separate/train/".format(s3_path)
test_inputs="{}/separate/test/".format(s3_path)

### 利用sagemaker pytorch进行训练

In [3]:
from sagemaker.pytorch import PyTorch

dprnn_estimator = PyTorch(entry_point='dprnn-train-deploy.py',
                            source_dir='code',
                            train_instance_type='ml.p3.2xlarge',
                            train_instance_count=1,
                            role=_role,
                            train_use_spot_instances=True, #save up to 90% training cost
                            train_max_wait=86400,
                            framework_version='1.5.1',
                            hyperparameters = {'epochs': 20})
dprnn_estimator.fit({'train': train_inputs,
                       'test': test_inputs})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-06-26 06:37:39 Starting - Starting the training job...
2020-06-26 06:37:43 Starting - Launching requested ML instances......
2020-06-26 06:38:47 Starting - Preparing the instances for training............
2020-06-26 06:41:05 Downloading - Downloading input data
2020-06-26 06:41:05 Training - Downloading the training image......
2020-06-26 06:42:10 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-26 06:42:10,841 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2020-06-26 06:42:10,863 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-26 06:42:13,879 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-26 06:42:14,198 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip 

### 将训练好的模型进行部署

In [15]:
# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = dprnn_estimator.deploy(instance_type='ml.g4dn.xlarge',
                                     initial_instance_count=1)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

In [ ]:
# 或者已有训练结果,创建endpoint
from sagemaker.pytorch.model import PyTorchModel
import time 

#修改为模型路径
model_path='s3://sagemaker-cn-northwest-1-969422986683/pytorch-training-2020-06-26-06-37-38-883/output/model.tar.gz' 
model_name='shouqiDPRNN'

sm_model = PyTorchModel(model_data=model_path,
                     role=_role,
                     name=model_name,
                        source_dir='code',
                    entry_point='code/dprnn-train-deploy.py',
                     py_version="py3",
                     framework_version="1.5.1",
                     sagemaker_session=sagemaker_session)
endpoint_name = f'endpoint-{model_name}-' \
                    + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

predictor = sm_model.deploy(endpoint_name=endpoint_name,
                         instance_type='ml.g4dn.xlarge',
                         initial_instance_count=1)


In [5]:
#或者已有endpoint restore predictor
import sagemaker
from sagemaker.pytorch.model import PyTorchPredictor

#修改为endpoint_name
endpoint_name='endpoint-shouqiDPRNN-2020-06-26-07-32-33'
predictor=PyTorchPredictor(endpoint_name, sagemaker_session=sagemaker_session)

### 在部署好的模型上进行推理

In [3]:
instance_time = 10

filename = './test.wav'

def wav_clip(filename,instance_time=instance_time,sample_rate=sample_rate):
    T = instance_time*sample_rate
    fs, x = scipy.io.wavfile.read(filename)
    emp = np.zeros(T)
    if x.shape[0]<=T:
        emp[:x.shape[0]] = x
        return emp
    else:
        return x[:T]
    
x=wav_clip(filename)
x.shape

(160000,)

In [6]:
x, fs = preprocess_wav(filename)
s_x = instance_time * fs
l_x = len(x)
s1 = np.array([])
s2 = np.array([])
gl_i = 0
for start in range(0, l_x-s_x, s_x):
    flat_estimate = separate_process(x[start:start+s_x], predictor)
    s1 = np.concatenate([s1, flat_estimate[0]])
    s2 = np.concatenate([s2, flat_estimate[1]])
    gl_i = start+s_x
if gl_i < l_x:
    flat_estimate = separate_process(x[gl_i:], predictor)
    s1 = np.concatenate([s1, flat_estimate[0]])
    s2 = np.concatenate([s2, flat_estimate[1]])
write(s1, './complete_split_s1.wav')
write(s2, './complete_split_s2.wav')
print('saved')

saved
